In [5]:
import sys
sys.path.append('..')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from interface.load_data import (
    load_data_fault_free_test,
    load_data_fault_free_train,
    load_data_faulty_test,
    load_data_faulty_train,
    load_data_fault_free_test_select

)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [6]:
from keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping

In [7]:
faulty_train = load_data_faulty_train()
faulty_train.shape

(5000000, 55)

In [8]:
mask = [i*10 for i in range(int(faulty_train.shape[0]/10))]

faulty_train_w = faulty_train.iloc[mask]
faulty_train_w.shape

(500000, 55)

In [ ]:
#list = [slice() for i in range]
#np.r_list

In [9]:
X = faulty_train.drop(columns=['faultNumber', 'simulationRun', 'sample'])
y = faulty_train.faultNumber

In [10]:
X_w = faulty_train_w.drop(columns=['faultNumber', 'simulationRun', 'sample'])
y_w = faulty_train_w.faultNumber

In [53]:
X_feat_select = faulty_train_w[['xmeas_6', 'xmeas_18', 'xmeas_7', 'xmeas_13']]
y_feat_select = faulty_train_w.faultNumber

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_w, y_w, test_size=0.3)

In [ ]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

In [ ]:
n_simulation = 500
len_simulation = 10000
n_per_sample = 50
n_fault = 20
n_features = 52

In [ ]:
X_train_reshaped = X_train_scaled.values.reshape(round(len_simulation*0.7), n_per_sample, n_features)
y_train_reshaped = y_train.values.reshape(round(len_simulation*0.7), n_per_sample)[:, 0]

X_test_reshaped = X_test_scaled.values.reshape(round(len_simulation*0.3), n_per_sample, n_features)
y_test_reshaped = y_test.values.reshape(round(len_simulation*0.3), n_per_sample)[:, 0]

In [61]:
print(X_train_reshaped.shape)
print(y_train_reshaped.shape)

(7000, 50, 4)
(7000,)


In [ ]:
model = Sequential([
    Input(shape=(50, 52)),

    layers.Conv1D(64, kernel_size=7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(128, kernel_size=5, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(512, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Flatten(),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),

    layers.Dense(21, activation='softmax')
])

In [63]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [67]:
callbacks = [EarlyStopping(patience=15)]

history = model.fit(
    X_train_reshaped, y_train_reshaped,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7404 - loss: 0.8003
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7616 - loss: 0.7465
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7944 - loss: 0.6458
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8077 - loss: 0.5907
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8286 - loss: 0.5331
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8371 - loss: 0.4988
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8616 - loss: 0.4552
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8680 - loss: 0.4160
Epoch 9/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8810 - loss: 0.3925
Epoch 10/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8886 - loss: 0.3669
Epoch 11/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8929 - loss: 0.3528
Epoch 12/100
219/219 ━━━━━━━━━

In [68]:
model.evaluate(X_test_reshaped, y_test_reshaped)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1797 - loss: 5.5240


[5.523989677429199, 0.179666668176651]

In [ ]:
#list = [slice() for i in range]
#np.r_list